In [ ]:
import os
import gc
import re
import sys
import json
import time
import random
import requests
import argparse
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
import scipy.io as sio
from tqdm import tqdm
from pathlib import Path
from collections import Counter
from collections import defaultdict
from logging import getLogger
from ast import Global
from functools import reduce

from ALDI import *
from utils import *
import daisy
from daisy.utils.config import init_seed, init_config, init_logger
from daisy.utils.metrics import MAP, NDCG, Recall, Precision, HR, MRR
from daisy.utils.utils import get_history_matrix, get_ur, build_candidates_set, ensure_dir, get_inter_matrix
from daisy.model.MFRecommender import MF

from daisy.model.LightGCNRecommender import LightGCN
from daisy.utils.metrics import calc_ranking_results

In [ ]:
model_config = {
    'mostpop': MostPop,
    'slim': SLiM,
    'itemknn': ItemKNNCF,
    'puresvd': PureSVD,
    'mf': MF,
    'fm': FM,
    'ngcf': NGCF,
    'neumf': NeuMF,
    'nfm': NFM,
    'multi-vae': VAECF,
    'item2vec': Item2Vec,
    'ease': EASE,
    'lightgcn': LightGCN,
}


config = init_config()
init_seed(config['seed'], config['reproducibility'])
init_logger(config)
logger = getLogger()
logger.info(config)
config['logger'] = logger

save_path = config['save_path'] + config['version']
ensure_dir(save_path)

file_path = save_path + f'{config["dataset"]}/'
ensure_dir(file_path)

saved_data_path = config['save_path'] + f'daisy_1/{config["dataset"]}/' + 'data/'
ensure_dir(saved_data_path)

saved_result_path = file_path + f'{config["algo_name"]}/'
ensure_dir(saved_result_path)

saved_model_path = saved_result_path + 'model/'
ensure_dir(saved_model_path)

saved_trd_path = saved_result_path + f'{trd_version}/'
ensure_dir(saved_trd_path)

saved_rec_path = saved_result_path + 'rec_list/'
ensure_dir(saved_rec_path)

saved_metric_path = saved_result_path + 'metric/'
ensure_dir(saved_metric_path)
config['res_path'] = saved_metric_path

ui_num = np.load(saved_data_path + 'ui_cate.npy')
config['user_num'] = ui_num[0]
config['item_num'] = ui_num[1]

print(f"user number: {config['user_num']}  item number: {config['item_num']}")

In [3]:
train_total_set = pd.read_csv(saved_data_path + 'train_total_set.csv', index_col=0)
test_u_all = np.load(saved_data_path + 'test_u_all.npy', allow_pickle=True)
test_ucands_all = np.load(saved_data_path + 'test_ucands_all.npy',allow_pickle=True)
test_ur_all = np.load(saved_data_path + 'test_ur_all.npy', allow_pickle=True)
data_last_stage = train_total_set
# result_df_all = pd.DataFrame()
warm_item_list_stage = []
cold_item_list_stage = []
new_item_list_stage = []
for stage in range(config['test_stage']):
    logger.info(f'begin test stage {stage}')
    # load test data
    test_u = test_u_all[stage]
    test_ur = test_ur_all[stage]
    test_ucands = test_ucands_all[stage]
    test_set = pd.read_csv(saved_data_path + f'test_set_{stage}.csv')
    # test_pd = pd.read_csv(saved_data_path + f'test_df_{stage}', index_col = 0)

    # update warm ，cold set and new items in this stage
    start_time = time.time()
    warm_item_list, cold_item_list = update_new_item(data_last_stage, config)
    warm_item_list_stage.append(warm_item_list)
    cold_item_list_stage.append(cold_item_list)
    # config['warm_item_list'] = warm_item_list
    # config['cold_item_list'] = cold_item_list
    new_item_list = list(set(test_set[config['IID_NAME']].unique()) - set(warm_item_list))
    new_item_list_stage.append(new_item_list)
    config['topk_list'] = [10, 20, 50]
    data_last_stage = pd.concat([data_last_stage, test_set])
    end_time = time.time()
    elapsed_time = end_time - start_time
    logger.info(f":finished update warm/cold_list: {elapsed_time:.6f} s")

23 Jan 09:57 INFO - begin test stage 0
23 Jan 09:57 INFO - :finished update warm/cold_list: 0.075160 s
23 Jan 09:57 INFO - begin test stage 1
23 Jan 09:57 INFO - :finished update warm/cold_list: 0.129882 s
23 Jan 09:57 INFO - begin test stage 2
23 Jan 09:57 INFO - :finished update warm/cold_list: 0.110933 s
23 Jan 09:57 INFO - begin test stage 3
23 Jan 09:57 INFO - :finished update warm/cold_list: 0.135777 s
23 Jan 09:57 INFO - begin test stage 4
23 Jan 09:57 INFO - :finished update warm/cold_list: 0.150264 s


In [ ]:
np.save(saved_data_path + 'warm_item_list_stage.npy', warm_item_list_stage)
np.save(saved_data_path + 'cold_item_list_stage.npy', cold_item_list_stage)
np.save(saved_data_path + 'new_item_list_stage.npy', new_item_list_stage)

In [4]:
from functools import reduce


def set_values_by_id(interaction_history, item_num):
    """
    Convert a user's interaction history into a fixed-length binary list.

    Args:
        interaction_history (list): List of item IDs that the user has interacted with.
        item_num (int): Total number of items.

    Returns:
        list: A binary list where 1 indicates interaction and 0 indicates no interaction.
    """
    # Use numpy's vectorized operation to check if each item is in the interaction history
    return np.isin(np.arange(item_num), interaction_history).astype(int).tolist()


def get_weight(warm_item_list, cold_item_list):
    """
    Compute weights for warm and cold items.

    Args:
        warm_item_list (list): List of warm item indices.
        cold_item_list (list): List of cold item indices.

    Returns:
        warm_weight (np.array): Weights for warm items (decreasing from warm_num to 1).
        cold_weight (np.array): Weights for cold items (increasing from 1 to warm_num-like values).
    """
    warm_num = len(warm_item_list)
    warm_weight = np.arange(warm_num, 0, -1)  # Weights decrease from warm_num to 1

    cold_num = len(cold_item_list)
    if cold_num > 1:
        # Weights increase from 1 to warm_num-like values
        cold_weight = 1 + np.arange(cold_num) * (warm_num - 1) / (cold_num - 1)
    else:
        cold_weight = np.array([1])  # If only one cold item, weight is 1

    return warm_weight, cold_weight


def get_user_hist_tgf(exp_list, warm_item_list, cold_item_list, warm_weight, cold_weight):
    """
    Calculate the Top-K Group Fairness (TGF) for a user based on their exposure distribution.

    Args:
        exp_list (list): List of exposure values for all items.
        warm_item_list (list): List of warm item indices.
        cold_item_list (list): List of cold item indices.
        warm_weight (np.array): Weights for warm items.
        cold_weight (np.array): Weights for cold items.

    Returns:
        float: TGF value, representing the fairness gap between warm and cold items for the user.
    """
    if np.sum(exp_list) == 0:
        return 0  # Return 0 if there is no exposure

    # Normalize exposure values
    exp_list = exp_list / np.sum(exp_list)

    # Extract exposure values for warm and cold items
    warm_exp_list = exp_list[warm_item_list]
    cold_exp_list = exp_list[cold_item_list]

    # Compute weighted exposure for warm and cold items
    warm_part = np.sum(warm_exp_list * warm_weight) / len(warm_item_list)
    cold_part = np.sum(cold_exp_list * cold_weight) / len(cold_item_list)

    # Compute TGF as the difference between warm and cold exposure
    user_tgf = warm_part - cold_part
    if user_tgf < 0:
        # Normalize TGF if it is negative
        user_tgf = user_tgf / (len(warm_item_list) / len(cold_item_list))
    return user_tgf


def calculate_nc(input_set, target_set):
    """
    Calculate the proportion of values in the input set that appear in the target set.

    Args:
        input_set (list): Input list of values.
        target_set (list): Target list of values.

    Returns:
        float: Proportion of values in the input set that appear in the target set.
    """
    if not input_set:
        return 0.0  # Return 0 if the input set is empty
    return len(set(input_set) & set(target_set)) / len(input_set)


def process_user_interaction_data(train_rl_set, warm_item_list, cold_item_list, config):
    """
    Process user interaction data to generate a DataFrame with user history, exposure lists, TGF, and NC.

    Args:
        train_rl_set (pd.DataFrame): Training set containing user-item interactions.
        warm_item_list (list): List of warm item indices.
        cold_item_list (list): List of cold item indices.
        config (dict): Configuration dictionary containing 'item_num'.

    Returns:
        pd.DataFrame: Processed DataFrame with user history, exposure lists, TGF, and NC.
    """
    # Group by user and generate user history lists
    user_rl_df = train_rl_set.groupby('user')['item'].apply(list).reset_index(name='user_history')

    # Generate exposure lists for each user
    item_range = np.arange(config['item_num'])
    user_rl_df['exp_list'] = user_rl_df['user_history'].apply(
        lambda x: np.isin(item_range, x).astype(int).tolist()
    )

    # Compute weights for warm and cold items
    warm_weight, cold_weight = get_weight(warm_item_list, cold_item_list)

    # Calculate TGF for each user
    user_rl_df['hist_tgf'] = user_rl_df['exp_list'].apply(
        lambda x: get_user_hist_tgf(np.array(x), warm_item_list, cold_item_list, warm_weight, cold_weight)
    )

    # Calculate NC for each user
    user_rl_df['hist_nc'] = user_rl_df['user_history'].apply(
        lambda x: calculate_nc(x, cold_item_list)
    )

    return user_rl_df


def get_common_users(dataframes):
    """
    Extract common 'user' values from multiple DataFrames.

    Args:
        dataframes (list): List of DataFrames.

    Returns:
        set: Set of common user IDs.
    """
    if not dataframes:
        return set()

    # Extract 'user' columns from all DataFrames
    all_users = [df['user'].values for df in dataframes if 'user' in df.columns]

    if not all_users:
        return set()

    # Use numpy's reduce to find the intersection of all user arrays
    return set(reduce(np.intersect1d, all_users))


def calculate_mse(list1, list2):
    """
    Calculate the Mean Squared Error (MSE) between two lists.

    Args:
        list1 (list): First list.
        list2 (list): Second list.

    Returns:
        float: MSE between the two lists.
    """
    return np.mean((np.array(list1) - np.array(list2)) ** 2)


def average_mse(array1, array2):
    """
    Calculate the average MSE between corresponding lists in two arrays, normalized to [0, 1].

    Args:
        array1 (list): First array containing five lists.
        array2 (list): Second array containing five lists.

    Returns:
        float: Normalized average MSE.
    """
    if len(array1) != len(array2):
        raise ValueError("Both arrays must have the same length")

    # Calculate MSE for each pair of corresponding lists
    mse_values = [calculate_mse(list1, list2) for list1, list2 in zip(array1, array2)]

    # Normalize MSE values to [0, 1]
    normalized_mse_values = [mse / 4 for mse in mse_values]

    # Calculate the average normalized MSE
    avg_mse = np.mean(normalized_mse_values)
    return avg_mse

In [5]:
# load test set - Ground-truth
test_set_list = []
for stage in range(config['test_stage']):
    test_set = pd.read_csv(saved_data_path + f'test_set_{stage}.csv')
    test_set_list.append(test_set)
# process_user_interaction_data(test_set, warm_item_list, cold_item_list, config)

In [7]:
# Get common users for all test stages
common_users = get_common_users(test_set_list)

In [10]:
np.save(saved_data_path + 'common_users.npy', common_users)

In [8]:
#calculate ground truth - TGF(Hu)
start_time = time.time()
mode = 'ground_truth'
hist_list = []
for stage in range(config['test_stage']):
    df = test_set_list[stage]
    df = df[df['user'].isin(common_users)]
    warm_item_list = warm_item_list_stage[stage]
    cold_item_list = cold_item_list_stage[stage]
    df_processed = process_user_interaction_data(df, warm_item_list, cold_item_list, config)
    df_processed.to_csv(saved_data_path + f'unf_{mode}_stage{stage}')
    hist_list.append(df_processed['hist_tgf'].to_list())
end_time = time.time()
elapsed_time = end_time - start_time
print(f":finished calculate TGF(Hu) for {config['dataset']}: {elapsed_time:.6f} s")

:finished update warm/cold_list: 6.962107 s


In [13]:
np.save(saved_data_path + 'hist_list_ground_truth.npy', hist_list)

In [ ]:
# Calculate TGF(Lu) for backbone models/baseline.
start_time = time.time()
for method in ['backbone', 'pd', 'pearson', 'cnif']:
    print(saved_metric_path)
    config['debias_method'] = method
    user_hist_list = []
    for stage in range(config['test_stage']):
        # Top-K recommendation results of backbone models/baseline
        df = pd.read_csv(saved_rec_path + f"{config['debias_method']}_{config['dataset']}_{config['algo_name']}_rec_df{stage}")
        df = df[df['user'].isin(common_users)]
        warm_item_list = warm_item_list_stage[stage]
        cold_item_list = cold_item_list_stage[stage]
        df_processed = process_user_interaction_data(df, warm_item_list, cold_item_list, config)
        df_processed.to_csv(saved_metric_path + f"unf_{config['debias_method']}_stage{stage}")
        user_hist_list.append(df_processed['hist_tgf'].to_list()) 
    np.save(saved_metric_path + f"hist_list_{config['debias_method']}.npy", user_hist_list)
end_time = time.time()
elapsed_time = end_time - start_time
print(f":finished calculate TGF(Lu) for {config['debias_method']}: {elapsed_time:.6f} s")

In [40]:
result_list = []
for method in ['backbone', 'pd', 'pearson', 'cnif']:
    config['debias_method'] = method
    user_hist_ground_truth = np.load(saved_data_path + 'hist_list_ground_truth.npy', allow_pickle=True) 
    user_hist_baseline = np.load(saved_metric_path + f"hist_list_{config['debias_method']}.npy", allow_pickle=True) 
    avg_mse = average_mse(user_hist_ground_truth, user_hist_baseline)
    result_list.append(avg_mse)
result_list

[0.06209650499226749,
 0.06728811082521705,
 0.0675830702855251,
 0.06079082819450367]

In [ ]:
# Calculate TGF(Lu) for FairAgent.
start_time = time.time()
start_time = time.time()
user_hist_list = []
for stage in range(config['test_stage']):
    # Top-K recommendation results of FairAgent
    df = pd.read_csv(saved_metric_path + f"fairagent_rec_results_fair_alpha0.0_beta0.0_gama0.1_stage{stage}.csv")
    df = df[df['user'].isin(common_users)]
    warm_item_list = warm_item_list_stage[stage]
    cold_item_list = cold_item_list_stage[stage]
    df_processed = process_user_interaction_data(df, warm_item_list, cold_item_list, config)
    df_processed.to_csv(saved_metric_path + f"fairagent_unf_stage{stage}")
    user_hist_list.append(df_processed['hist_tgf'].to_list()) 
np.save(saved_metric_path + f"fairagent_hist_list.npy", user_hist_list)
end_time = time.time()
elapsed_time = end_time - start_time
print(f":finished calculate TGF(Lu) for FairAget: {elapsed_time:.6f} s")

user_hist_ground_truth = np.load(saved_data_path + 'hist_list_ground_truth.npy') 
user_hist_baseline = np.load(saved_metric_path + f"fairagent_hist_list.npy") 
avg_mse = average_mse(user_hist_ground_truth, user_hist_baseline)
avg_mse